# Ikeda for many ships with quadratic contributions
The method developed in: ([01.03_ikeda_many_dev](06_ikeda/01.03_ikeda_many_dev.ipynb)) will now be attempted for many ships.
Instead of just calculating ikeda for one roll amplitude, it will now be calculated for 2 amplitudes to derive the quadratic part $B_2$ 


In [ ]:
# %load ../../imports.py
"""
These is the standard setup for the notebooks.
"""

%matplotlib inline
%load_ext autoreload
%autoreload 2

#from jupyterthemes import jtplot
#jtplot.style(theme='onedork', context='notebook', ticks=True, grid=False)

import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)
import numpy as np
import os
import matplotlib.pyplot as plt
#plt.style.use('paper')

#import data
import copy
from rolldecay.bis_system import BisSystem
from rolldecay import database
from mdldb.tables import Run

from sklearn.pipeline import Pipeline
from rolldecayestimators.transformers import CutTransformer, LowpassFilterDerivatorTransformer, ScaleFactorTransformer, OffsetTransformer
from rolldecayestimators.direct_estimator_cubic import EstimatorQuadraticB, EstimatorCubic
from rolldecayestimators.ikeda_estimator import IkedaQuadraticEstimator
import rolldecayestimators.equations as equations
import rolldecayestimators.lambdas as lambdas
from rolldecayestimators.substitute_dynamic_symbols import lambdify
import rolldecayestimators.symbols as symbols
import sympy as sp

from sklearn.metrics import r2_score
import rolldecay.paper_writing as paper_writing



In [ ]:
from pyscores2.indata import Indata
from pyscores2.runScores2 import Calculation
from pyscores2.output import OutputFile
from pyscores2 import TDPError
import pyscores2
from rolldecayestimators.ikeda import Ikeda, IkedaR
from rolldecayestimators.simplified_ikeda_class import SimplifiedIkeda
import subprocess
from rolldecayestimators import measure
import rolldecay

In [ ]:
df_all_sections_id = pd.read_csv('all_sections.csv', sep=';')
df_all_sections_id.head()

In [ ]:
section_groups=df_all_sections_id.groupby(by='loading_condition_id')

In [ ]:
loading_condition_ids = df_all_sections_id['loading_condition_id'].unique()
mask=pd.notnull(loading_condition_ids)
loading_condition_ids=loading_condition_ids[mask]

In [ ]:
df_rolldecay = database.load(rolldecay_table_name='rolldecay_quadratic_b', limit_score=0.99, 
                             exclude_table_name='rolldecay_exclude')

In [ ]:
mask=df_rolldecay['loading_condition_id'].isin(loading_condition_ids)
df=df_rolldecay.loc[mask].copy()

In [ ]:
df['BKB'].fillna(0, inplace=True)

In [ ]:
def add_cScores(sections):
    sections=sections.copy()
    sections['cScores']=sections['area']/(sections['b']*sections['t'])
    mask=sections['cScores']>1
    sections.loc[mask,'cScores']=1
    return sections

def cut_sections(sections, draught):
    sections=sections.copy()
    mask = sections['t']>draught
    sections.loc[mask,'t']=draught
    sections.loc[mask,'area']-=draught*sections['b'].max()  # Assuming rectangular shape
    return sections

def remove_duplicate_sections(sections):
    sections=sections.copy()
    mask=~sections['x'].duplicated()
    sections=sections.loc[mask]
    assert sections['x'].is_unique
    return sections

def too_small_sections(sections):
    sections=sections.copy()
    small = 0.1
    mask=sections['b']==0
    sections.loc[mask,'b']=small
    mask=sections['t']==0
    sections.loc[mask,'t']=small
    mask=sections['area']==0
    sections.loc[mask,'area']=small
    return sections

In [ ]:
from scipy.integrate import simps
def calculate_lcb(x, area, **kwargs):
    """
    Calculate lcb from AP
    """
    return simps(y=area*x,x=x)/np.trapz(y=area,x=x)

def calculate_dispacement(x, area, **kwargs):
    """
    Calculate displacement
    """
    return np.trapz(y=area,x=x)

In [ ]:
class DraughtError(ValueError): pass

def define_indata(row, sections, rho=1000,  g=9.81):
    
    indata = Indata()
    
    draught=(row.TA+row.TF)/2
    indata.draught=draught
    if draught<=sections['t'].max():
        sections = cut_sections(sections, draught)
    else:
        raise DraughtError('Draught is too large for sections')
    
    sections=add_cScores(sections)
    
    indata.cScores=np.array(sections['cScores'])
    indata.ts=np.array(sections['t'])
    indata.bs=np.array(sections['b'])
    indata.zbars=np.zeros_like(sections['b'])  # Guessing...
        
    beam=sections['b'].max()
    indata.lpp=sections['x'].max()-sections['x'].min()
    #indata.displacement=row.Volume
    indata.displacement=calculate_dispacement(**sections)
    
    indata.g=g
    indata.kxx=row.KXX
    indata.kyy=row.lpp*0.4
    lcb=calculate_lcb(x=sections['x'], area=sections['area'])
    indata.lcb=lcb-row.lpp/2
    indata.lpp=row.lpp
    indata.projectName='loading_condition_id_%i' % row.loading_condition_id
    
    indata.rho=rho
    indata.zcg=row.kg-draught
    #indata.waveFrequenciesMin=0.2
    #indata.waveFrequenciesMax=0.5
    #indata.waveFrequenciesIncrement=0.006
    w=row.omega0/np.sqrt(row.scale_factor)
    indata.waveFrequenciesMin=w*0.5
    indata.waveFrequenciesMax=w*2.0
    N=40
    indata.waveFrequenciesIncrement=(indata.waveFrequenciesMax-indata.waveFrequenciesMin)/N
    indata.runOptions["IE"].set_value(1)
    
    return indata,sections

In [ ]:
def create_ikeda(row, indata, output_file, fi_a):

    w = row.omega0
    scale_factor=row.scale_factor
    V = row.ship_speed*1.852/3.6/np.sqrt(scale_factor)
    R = 0.01*row.beam/scale_factor
    lBK=row.BKL/scale_factor
    bBK=row.BKB/scale_factor
    ikeda = Ikeda.load_scoresII(V=V, w=w, fi_a=fi_a, indata=indata, output_file=output_file, 
                                scale_factor=scale_factor, lBK=lBK, bBK=bBK)
    
    ikeda.R = R
    return ikeda

In [ ]:
def calculate_ikeda(ikeda):

    output = pd.DataFrame()
    output['B_44_hat']   = ikeda.calculate_B44()
    output['B_W0_hat']   = float(ikeda.calculate_B_W0())
    output['B_W_hat']    = float(ikeda.calculate_B_W())
    output['B_F_hat']    = ikeda.calculate_B_F()
    output['B_E_hat']    = ikeda.calculate_B_E()
    output['B_BK_hat']   = ikeda.calculate_B_BK()
    output['B_L_hat']    = float(ikeda.calculate_B_L())
    output['Bw_div_Bw0'] = float(ikeda.calculate_Bw_div_Bw0())
    return output

In [ ]:
def get_B_1_B2(s1,s2,fi_as:np.ndarray):    
    
    # Derive linear and quadratic part for all components:
    
    if isinstance(s1,pd.Series):
        result=pd.Series(name=row.name)
        columns = s1.index
    elif isinstance(s1,pd.DataFrame):
        result=pd.DataFrame()
        columns = s1.columns
    else:
        raise ValueError('s1 must be pd.Series or pd.DataFrame')
    
    x = fi_as*8*row.omega0/(3 * np.pi)
    B_2 = (s2 - s1) / (x[1] - x[0])
    B_1 = s1 - B_2 * x[0]
    
    # Save all of the component as one linear term: _1 and a quadratic term: _2
    
    
    for key in columns:
        new_name_1 = '%s_1' % key
        result[new_name_1] = s1[key]

        new_name_2 = '%s_2' % key
        result[new_name_2] = s2[key]
        
    return result

In [ ]:
results = pd.DataFrame()
fi_as = np.deg2rad([1,10])
for run_name, row in df.iterrows():
    loading_condition_id=row['loading_condition_id']
    sections = section_groups.get_group(loading_condition_id)
    sections=remove_duplicate_sections(sections)
    sections=too_small_sections(sections)
    
    try:
        indata,sections_ = define_indata(row, sections)
    except DraughtError as e:
        print('Draught is too large for sections, this loading condition is skipped.')
        continue

        
    save_name='%s.in' % row.loading_condition_id
    save_path=os.path.join('scores2',save_name)
    indata.save(save_path)
    
    calculation = Calculation(outDataDirectory='scores2/result')
    
    # Run scoresII:
    try:
        calculation.run(indata=indata, b_div_t_max=None, timeout=1.0)
    except TDPError:
        print('Dissregarding the TDPError')
        continue
    except pyscores2.LcgError as e:
        print('Disregarded')
        print(e)
        continue
    except subprocess.TimeoutExpired:
        print('Disregarded, scoresII got stuck...')
        continue
        
    output_file = OutputFile(filePath=calculation.outDataPath)
    
    
    ikeda = create_ikeda(row=row, indata=indata, output_file=output_file, fi_a=fi_as)
    result_datas = calculate_ikeda(ikeda)  # DataFrame with two roll amplitudes
    
    # Derive linear and quadratic part for all components:
    s1=result_datas.iloc[0]
    s2=result_datas.iloc[1]
    result = get_B_1_B2(s1=s1, s2=s2, fi_as=fi_as)
    
    
    results=results.append(result)
    

In [ ]:
results['id']=results.index

In [ ]:
results

In [ ]:
results.head()

In [ ]:
df.head()

## Make comparison with model tests

In [ ]:
df2 = df.copy()
scale_factor = df2['scale_factor']
df2['Volume']/=(scale_factor**3)
df2['Disp']=df2['Volume']
df2['beam']/=scale_factor
df2['lpp']/=scale_factor
df2['kg']/=scale_factor
df2['BKB']/=scale_factor
df2['BKL']/=scale_factor




In [ ]:
df['Disp']=df2['Volume']
results['Disp']=df2.loc[results.index,'Disp']
results['omega0']=df2.loc[results.index,'omega0']

#df_compare = measure.linearized_matrix(df_rolldecay=df2, df_ikeda=results, do_hatify=False)
#mask=df_compare['B_e_hat_ikeda'].notnull()
#df_compare=df_compare.loc[mask].copy()

In [ ]:
def linearize_model_tests(df_rolldecay, phi_as = np.deg2rad(np.linspace(1,10,10)), g=9.81, rho=1000):

    df = pd.DataFrame()
    
    for phi_a in phi_as:
        df_ = measure.linearize_model_test(phi_a=phi_a, df_rolldecay=df_rolldecay, g=g, rho=rho)
        df_['phi_a']=phi_a
        df =df.append(df_, ignore_index=True)
        
    return df

def linearize_ikedas(df_ikeda, phi_as = np.deg2rad(np.linspace(1,10,10)), g=9.81, rho=1000,
                     components = ['B_44_hat', 'B_F_hat', 'B_W_hat', 'B_E_hat', 'B_BK_hat', 'B_L_hat']):

    df = pd.DataFrame()
    
    
    for phi_a in phi_as:
        df_ = measure.linearize_si(phi_a=phi_a, df_ikeda=df_ikeda, do_hatify=False, components=components)
        df_['phi_a']=phi_a
        df =df.append(df_, ignore_index=True)
        
    return df

In [ ]:
df_rolldecays=linearize_model_tests(df_rolldecay=df2)
df_ikedas=linearize_ikedas(df_ikeda=results)
df_ikedas.dropna(subset=['B_e_hat'], inplace=True)

In [ ]:
df_compare = pd.merge(left=df_rolldecays, right=df_ikedas, how='inner', left_on=('id','phi_a'), right_on=('id','phi_a'), suffixes=('','_ikeda'))

In [ ]:
fig,ax=plt.subplots()
df_compare.plot(x='B_e_hat', y='B_e_hat_ikeda', ax=ax, style='o', alpha=0.05, ms=20, markeredgewidth=0)

In [ ]:
r2_score(y_true=df_compare['B_e_hat'], y_pred=df_compare['B_e_hat_ikeda'])

## Also run Simplified Ikeda for comparison

In [ ]:
def calculate_si(si):

    output = pd.DataFrame()
    output['B_44_hat'] = si.calculate_B44()
    output['B_W0_hat'] =si.calculate_B_W0()
    output['B_W_hat'] =si.calculate_B_W()
    output['B_F_hat'] =si.calculate_B_F()
    output['B_E_hat'] =si.calculate_B_E()
    output['B_BK_hat'] =si.calculate_B_BK()
    output['B_L_hat'] =si.calculate_B_L()
    output['Bw_div_Bw0'] =si.calculate_Bw_div_Bw0()
    return output

In [ ]:
inputs_si=pd.DataFrame()
inputs_si['w']=df['omega0']  # Already model scale
scale_factor=df['scale_factor']
inputs_si['V']=df['ship_speed']*1.852/3.6/np.sqrt(scale_factor)
inputs_si['beam']=df['beam']/scale_factor
inputs_si['lpp']=df['lpp']/scale_factor
inputs_si['kg']=df['kg']/scale_factor
inputs_si['volume']=df['Volume']/(scale_factor**3)
draught=(df['TA']+df['TF'])/2
inputs_si['draught']=draught/scale_factor
inputs_si['A0']=df['A0']
inputs_si['lBK']=df['BKL']/scale_factor
inputs_si['bBK']=df['BKB']/scale_factor

inputs_si_1=inputs_si.copy()
inputs_si_2=inputs_si.copy()


inputs_si_1['fi_a']=fi_as[0]
si_1 = SimplifiedIkeda(**inputs_si_1)
results_si_1 = calculate_si(si_1)

inputs_si_2['fi_a']=fi_as[1]
si_2 = SimplifiedIkeda(**inputs_si_2)
results_si_2 = calculate_si(si_2)

results_si=get_B_1_B2(s1=results_si_1, s2=results_si_2, fi_as=fi_as)
results_si.index=df.index

In [ ]:
results_si['Disp']=df2.loc[results_si.index,'Disp']
results_si['omega0']=df2.loc[results_si.index,'omega0']
results_si['id']=results_si.index
#df_compare_si = measure.linearized_matrix(df_rolldecay=df2, df_ikeda=results_si, do_hatify=False, suffixes=('','_si'))
df_sis=linearize_ikedas(df_ikeda=results_si)
df_sis.dropna(subset=['B_e_hat'], inplace=True)


In [ ]:
df_compare_ikeda_si = pd.merge(left=df_ikedas, right=df_sis, how='inner', left_on=('id','phi_a'), right_on=('id','phi_a'), suffixes=('_ikeda','_si'))
df_compare_all = pd.merge(left=df_rolldecays, right=df_compare_ikeda_si, how='inner', left_on=('id','phi_a'), right_on=('id','phi_a'), suffixes=('','_ikedas'))

In [ ]:
fig,ax=plt.subplots()
df_compare_all.plot(x='B_e_hat', y=['B_e_hat_ikeda','B_e_hat_si'], ax=ax, style='o', alpha=0.40, ms=5, markeredgewidth=0)

ax.set_xlabel(r'$\hat{B_{44}}$ (Ikeda)')
ax.set_ylabel(r'$\hat{B_{44}}$ (SI)')

xlim = ax.get_xlim()
ylim = ax.get_ylim()
lim = np.max([xlim[1],ylim[1]])
ax.set_xlim(0,lim)
ax.set_ylim(0,lim)
ax.plot([0,lim],[0,lim],'r-')

ax.grid(True)
ax.set_aspect('equal', 'box')
ax.get_legend().remove()

<a id="model_si_ikeda"></a>

In [ ]:
size=3.5
with plt.style.context('paper'):
    fig,ax=plt.subplots()
    fig.set_size_inches(size,size)
    
    df_compare_all.plot(x='B_e_hat', y='B_e_hat_si', ax=ax, style='+', 
                    label=r'$\hat{B}_e^{SI}$')

    df_compare_all.plot(x='B_e_hat', y='B_e_hat_ikeda', ax=ax, style='.', 
                    label=r'$\hat{B}_e^{Ikeda}$')
    
    ax.set_xlabel(r'$\hat{B}_e^{Model}$')
    ax.set_ylabel(r'$\hat{B}_e$')
    
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    lim = np.max([xlim[1],ylim[1]])
    ax.set_xlim(0,lim)
    ax.set_ylim(0,lim)
    #ax.set_title('Total roll damping for Ikeda, Simplified Ikeda and model tests')
    ax.plot([0,lim],[0,lim],'r-')
    
    ax.grid(True)
    ax.set_aspect('equal', 'box')
    ax.legend();
    paper_writing.save_fig(fig=fig, name='si_ikeda_model')

In [ ]:
df_compare_all['B_44_fraction_si_ikeda'] = df_compare_all['B_e_hat_si']/df_compare_all['B_e_hat_ikeda']

fig,ax=plt.subplots()
df_compare_all.plot(x='B_e_hat', y='B_44_fraction_si_ikeda', ax=ax, style='.')
ax.set_xlabel(r'$\hat{B_{44}}$')
ax.set_ylabel(r'$\frac{B_{44}(SI)}{B_{44}(Ikeda)}$')

ax.get_legend().remove()

In [ ]:
r2_score(y_true=df_compare_all['B_e_hat'], y_pred=df_compare_all['B_e_hat_ikeda'])

In [ ]:
r2_score(y_true=df_compare_all['B_e_hat'], y_pred=df_compare_all['B_e_hat_si'])

## Investigating the residuals

In [ ]:
def calculate_residuals(suffix_true='_ikeda', suffix_prediction='_si'):
    prefixes = ['B_e_hat',
                #'B_W0_hat_e',
                'B_W_hat_e',
                'B_F_hat_e',
                'B_E_hat_e',
                'B_BK_hat_e',
                'B_L_hat_e', 
                #'Bw_div_Bw0',
               ]
    
    for prefix in prefixes:
        residual_name = '%s_residual%s%s' % (prefix, suffix_prediction, suffix_true)
        name_true='%s%s' % (prefix, suffix_true)
        name_prediction='%s%s' % (prefix, suffix_prediction)
        
        df_compare_all[residual_name] = df_compare_all[name_prediction] - df_compare_all[name_true]



In [ ]:
df_compare_all.head()

In [ ]:
calculate_residuals()
df_compare_all['B_e_hat_residual_si_model'] = df_compare_all['B_e_hat_si'] - df_compare_all['B_e_hat']

In [ ]:
import seaborn as sns; 
#sns.set_theme()

<a id="residuals"></a>

In [ ]:
g=9.81
df_compare_all['draught']=((df_compare_all['TA'] + df_compare_all['TF'])/2)/df_compare_all['scale_factor']
df_compare_all['OG']=df_compare_all['draught']-df_compare_all['kg']
df_compare_all['beam/draught']=df_compare_all['beam']/df_compare_all['draught']
df_compare_all['V']=df_compare_all['ship_speed']*1.852
df_compare_all['Fn']=df_compare_all['V']/(np.sqrt(df_compare_all['lpp']*g))
df_compare_all[r'OG/d']=df_compare_all['OG']/df_compare_all['draught']
df_compare_all[r'LBK/Lpp']=df_compare_all['BKL']/df_compare_all['lpp']
df_compare_all[r'BBK/beam']=df_compare_all['BKB']/df_compare_all['beam']
df_compare_all['omega_hat']=lambdas.omega_hat(beam=df_compare_all['beam'], g=g, omega0=df_compare_all['omega0'])

df_compare_all['Cb']=df_compare_all['Volume']/(df_compare_all['lpp']*df_compare_all['beam']*df_compare_all['draught'])
interesting=['Cb','A0','OG/d','LBK/Lpp','BBK/beam','omega_hat',r'beam/draught', 'Fn']
#sns.lmplot(data=df_compare,y='B_44_hat_residual_si_ikeda', x=interesting, aspect=0.6);
sns.pairplot(df_compare_all,y_vars='B_e_hat_residual_si_ikeda', x_vars=interesting, aspect=0.6);

In [ ]:
corr = df_compare_all[interesting+['B_e_hat_residual_si_ikeda']].corr().abs()

fig,ax=plt.subplots()
corr.plot(y='B_e_hat_residual_si_ikeda', kind='bar', ax=ax);
ax.set_title('Absolut correlation coefficient for the error');

In [ ]:
with plt.style.context('paper'):
    
    y='B_e_hat_residual_si_model'
    
    ylabel=r'$\hat{B}_{e}^{SI}-\hat{B}_{e}^{Model}$'
    fig,ax=plt.subplots()
    fig.set_size_inches(size,size)
    
    df_compare_all.plot(x=r'beam/draught',y=y, ax=ax, style='.')
    
    
    ax.set_ylabel(ylabel)
    ax.set_xlabel(r'$\frac{beam}{T}$ [-]')
    ax.set_ylabel(ylabel)
    ax.grid(True)
    ax.get_legend().remove()
    paper_writing.save_fig(fig, name='beam_T_residual')

In [ ]:
import seaborn as  sns

g = sns.jointplot(x=r'beam/draught', y='B_e_hat_residual_si_model', data=df_compare_all,
                  kind="reg", truncate=False,
                  color="m", height=7)

In [ ]:
labels={
'Cb' : r'$C_b$ [-]',
r'beam/draught' : r'$\frac{beam}{T}$ [-]',
r'OG/d' : r'$\frac{\overline{OG}}{T}$ [-]',
'A0' : r'$A_{0}$ [-]',
r'BBK/beam' : r'$\frac{BK_B}{beam}$ [-]',
r'LBK/Lpp' : r'$\frac{BK_L}{L_{pp}}$ [-]',
r'omega_hat' : r'$\hat{\omega}$ [-]',
r'fi_a' : r'$\phi_a$ [rad]',
r'Fn' : r'$F_n$ [-]',
    
}

with plt.style.context('paper'):
    fig,ax=plt.subplots()
    fig.set_size_inches(size,size)
    
    df_compare_all['B_e_hat_residual_si_model_abs'] = df_compare_all['B_e_hat_residual_si_model'].abs()
    
    for y in interesting:
        
        df_=df_compare_all.sort_values(by=y)
        y_=df_[y].abs()
        y_-=y_.min()
        
        ax.plot(df_['B_e_hat_residual_si_model_abs'], y_, '.', label=labels[y])
    
    
    ax.set_xlabel(r'$|\hat{B}_{e}^{SI}-\hat{B}_{e}^{Model}|$')
    ax.set_ylabel('change')

    ax.grid(True)
    ax.legend(loc='upper center', bbox_to_anchor=(1.30, 0.8),
          ncol=1)
    
    paper_writing.save_fig(fig, name='parameter_residual')

## Comparing damping contributions SI vs. model test error

In [ ]:
suffix_true='_ikeda'
suffix_prediction='_si'
prefixes = [
    'B_W_hat_e_si',
    'B_F_hat_e_si',
    'B_E_hat_e_si',
    'B_BK_hat_e_si',
    'B_L_hat_e_si',]

labels=[
    r'$\hat{B_{W}}$',
    r'$\hat{B_{F}}$',
    r'$\hat{B_{E}}$',
    r'$\hat{B_{BK}}$',
    r'$\hat{B_{L}}$',
    ]

lim = np.max([df_compare_all['B_e_hat_ikeda'].max(),
              df_compare_all['B_e_hat_si'].max(),
             ])
with plt.style.context('paper'):
    
    fig,ax=plt.subplots()
    fig.set_size_inches(size,size)

        
        
    df_compare_all.plot(x='B_e_hat_residual_si_model_abs', y=prefixes, style='.', label=labels, ax=ax)
    ax.set_xlabel(r'$|\hat{B}_{e}^{SI}-\hat{B}_{e}^{Model}|$')
        
    ax.legend()
    ax.grid(True)
    ax.set_ylabel(r'$\hat{B}$ [-]')
    
    # (depricated) paper_writing.save_fig(fig, name='component_residual')  

In [ ]:
suffix_true='_ikeda'
suffix_prediction='_si'
prefixes = [
    'B_W_hat_e',
    'B_F_hat_e',
    'B_E_hat_e',
    'B_BK_hat_e',
    'B_L_hat_e',]

x_labels=['%s_ikeda' % label for label in prefixes]
y_labels=['%s_si' % label for label in prefixes]

fig,ax=plt.subplots()
for prefix in prefixes:
    x='%s_ikeda' % prefix
    y='%s_si' % prefix
    
    df_compare_all.plot(x=x, y=y, label=prefix, ax=ax, style='.')


<a id="B_W_speed_factor_residual"></a>

In [ ]:
parts=[
'B_W_hat_e_si',
'B_F_hat_e_si',
'B_E_hat_e_si',
'B_BK_hat_e_si',
'B_L_hat_e_si',
]

df_compare_all['B_e_hat_residual_si_model_abs'] = df_compare_all['B_e_hat_residual_si_model'].abs()
corr = df_compare_all[parts+['B_e_hat_residual_si_model_abs']].corr().abs()
corr.drop(index='B_e_hat_residual_si_model_abs', inplace=True)

fig,ax=plt.subplots()
corr.plot(y='B_e_hat_residual_si_model_abs', kind='bar', ax=ax);
ax.set_title('Absolut correlation coefficient for the error');
ax.legend().remove()

## Save to table

In [ ]:
mean_='$R^2$'
number_of_points = 'Number of points'

validation = pd.DataFrame(columns=[mean_, number_of_points])

s = pd.Series(name='Ikeda')
s[mean_] = r2_score(y_true=df_compare_all['B_e_hat'], y_pred=df_compare_all['B_e_hat_ikeda'])
s[number_of_points] = int(len(df_compare_all))
validation=validation.append(s)

s = pd.Series(name='SI no limits')
s[mean_] = r2_score(y_true=df_compare_all['B_e_hat'], y_pred=df_compare_all['B_e_hat_si'])
s[number_of_points] = int(len(df_compare_all))
validation=validation.append(s)

validation[number_of_points]=validation[number_of_points].astype(int)

In [ ]:
validation

In [ ]:
latex = validation.to_latex(float_format='%0.2f', na_rep='')
name='si_ikeda_validation'
file_path = os.path.join(rolldecay.equations_path,name)

# (Uncomment this one if you want to regenerate this table)
paper_writing.save_table(file_path=file_path, tabular_tex=latex, label='tab:si_ikeda_validation', 
           caption='Validation of SI and Ikeda')

In [ ]:
len(df_compare_all['id'].unique())

In [ ]:
len(df_compare_all['loading_condition_id'].unique())

In [ ]:
len(df_compare_all['model_number'].unique())

In [ ]:
#df_compare_all['draught']=(df_compare['TA'] + df_compare['TF'])/2

df_compare_all['CB'] = df_compare_all['Volume']/(df_compare_all['lpp']*df_compare_all['beam']*df_compare_all['draught'])
df_compare_all['g'] = 9.81
df_compare_all[r'B/d']=df_compare_all['beam']/df_compare_all['draught']
df_compare_all['OG']=df_compare_all['draught']-df_compare_all['kg']
df_compare_all[r'OG/d']=df_compare_all['OG']/df_compare_all['draught']
df_compare_all['CMID']=df_compare_all['A0']
df_compare_all[r'bBk/B']=df_compare_all['BKB']/df_compare_all['beam']
df_compare_all[r'lBk/LPP']=df_compare_all['BKL']/df_compare_all['lpp']
df_compare_all['OMEGA_hat']=lambdas.omega_hat(beam=df_compare_all['beam'], g=df_compare_all['g'], omega0=df_compare_all['omega0'])

In [ ]:
from rolldecayestimators.simplified_ikeda import limits_kawahara
limits_kawahara=pd.Series(limits_kawahara)

In [ ]:
def cut(df, limits, item):
    df=df.copy()
    mask = ((df[item] >= limits[0]) & (df[item] <= limits[1])) | (df[item]==0)
    df=df.loc[mask].copy()
    return df

def plot_select(df_compare, limits_used):
    
    df_compare_within = df_compare.copy()

    fig,axes=plt.subplots(nrows=len(limits_used))
    fig.set_size_inches(15,12)
    
    for (item, limits),ax in zip(limits_used.items(),axes):
        
        n_bins = 20
        bins=np.linspace(df_compare[item].min(), df_compare[item].max(),n_bins)
        df_compare.hist(item, ax=ax, bins=bins, color='gray')
        
        df_compare_within = cut(df=df_compare_within, limits=limits, item=item)
        df_compare_within.hist(item, ax=ax, bins=bins, color='blue')
        ax.set_xlabel(item)
        ax.set_title('')
        
        ylims = ax.get_ylim()
        ax.fill_between(limits, [ylims[1],ylims[1]], y2=0, color='green', alpha=0.5, label='valid')
        
    plt.tight_layout()
    return df_compare_within

In [ ]:
df_ = plot_select(df_compare_all, limits_kawahara)

In [ ]:
len(df_)